In [1]:
import ai_edge_torch


2024-09-20 17:36:30.387344: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-20 17:36:30.396063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1726875390.406059   47829 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1726875390.409048   47829 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 17:36:30.419670: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import torchvision.transforms as transforms
from PIL import Image
import numpy
import torch
import torchvision
import monai
import os

In [3]:
# Load the image
image_path = 'cat.jpeg'
image = Image.open(image_path).convert('RGB').resize((256,256))
preprocess = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image_tensor = preprocess(image).unsqueeze(0)

In [4]:
densenet = getattr(monai.networks.nets, 'densenet121')
model = densenet(spatial_dims=2,
                    in_channels=3,
                    out_channels=3,
                    dropout_prob=float(0.2),
                    pretrained=True)

In [5]:
# Load the weights from the file
weights_path = '../best_metric_model.pth'
state_dict = torch.load(weights_path)
model.load_state_dict(state_dict)

/tmp/ipykernel_47829/1251356623.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path)


<All keys matched successfully>

In [6]:
model.eval()

DenseNet121(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (dropout): Dropout2d(p=0.2, inplace=False)
        )
      )
      (denselayer2): _DenseLayer(
  

In [7]:
torch_output = model(image_tensor)
sample_inputs = (image_tensor,)

In [8]:
torch_output[0, 0:3]

tensor([ 8.1829, -2.2875, -6.3397], grad_fn=<SelectBackward0>)

In [9]:
import os

In [10]:
os.environ['PJRT_DEVICE'] = 'CPU'

In [11]:
edge_model = ai_edge_torch.convert(model.eval(), sample_inputs)

I0000 00:00:1726875416.077153   47829 cpu_client.cc:467] TfrtCpuClient created.
I0000 00:00:1726875416.332138   47829 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13937 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


INFO:tensorflow:Assets written to: /tmp/tmpc0zaehuv/assets


INFO:tensorflow:Assets written to: /tmp/tmpc0zaehuv/assets
W0000 00:00:1726875422.045162   47829 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1726875422.045179   47829 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-09-20 17:37:02.045630: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpc0zaehuv
2024-09-20 17:37:02.051778: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-20 17:37:02.051789: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpc0zaehuv
I0000 00:00:1726875422.113361   47829 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2024-09-20 17:37:02.122962: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-20 17:37:02.647145: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpc0zaehuv
2024-09-20 17:37:02.773

In [12]:
edge_output = edge_model(*sample_inputs)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [13]:
if (numpy.allclose(
    torch_output.detach().numpy(),
    edge_output,
    atol=1e-5,
    rtol=1e-5,
)):
    print("Inference result with Pytorch and TfLite was within tolerance")
else:
    print("Something wrong with Pytorch --> TfLite")


Inference result with Pytorch and TfLite was within tolerance


In [14]:
edge_model.export('modelthirtysix.tflite')
